In [1]:
using JuMP
using Gurobi
using DataFrames
using CSV
using LinearAlgebra
using Plots
using Statistics

In [11]:
function ridge(X, y, T)
    # extract problem dimensions from distance matrix and verify coherence of input data
    N,P  = size(X)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 30)
    # VARIABLES
    # Coefficients
    @variable(model, B[1:P])
    @variable(model, A[1:N-T,1:P])
    
    #Resource allocation constraint
    #@constraint(model, enough_employees[t=1:T, d=1:M],
            #sum(A[t, j]*x[d,j]  for j = 1:J) >= D[d, t])
    
    # OBJECTIVE
    @objective(
        model, Min, sum(sum((y[i]-sum(X[i,p]*(B[j]+A[i,p] for p=1:P))^2 for i=s:s+T) for s=1:N-T))
    )
    
    optimize!(model)
    
    return model, objective_value(model), B, A
end

ridge (generic function with 1 method)

In [9]:
X = convert(Matrix,CSV.File("X_test_adaptive.csv"; header=true) |> DataFrame!)[:,2:8];
y = convert(Matrix,CSV.File("y_test_speed.csv"; header=true) |> DataFrame!)[:,2];
#X = D1[:,1:4:72]

In [10]:
y

8494-element Array{Float64,1}:
 6.174999999999999
 6.2749999999999995
 6.15
 5.5
 6.575000000000001
 6.55
 5.9
 5.15
 3.7249999999999996
 3.175
 2.55
 2.475
 2.45
 ⋮
 3.9716666666666662
 3.651666666666667
 4.571666666666667
 5.156666666666668
 6.586666666666669
 6.991666666666666
 5.9433333333333325
 5.066666666666666
 5.308333333333333
 5.326666666666669
 6.835000000000002
 6.477419354838708

In [12]:
m, o, B, A = ridge(X, y, 3)

LoadError: [91mGurobi Error 10009: Failed to obtain a valid license[39m